# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [4]:
# импорт библиотек

import wikipedia
import numpy as np

In [8]:
# возьмем пять основных романских языков

langs = ['es', 'it', 'fr', 'pt', 'ro']

In [11]:
# функция загрузки статьи

def load_with_disambigution(page):
    try:
        p = wikipedia.page(page)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = wikipedia.page(random_option)
    return p

In [12]:
# функция загрузки n рандомных статец для некоторого языка

def get_texts_for_lang(lang, n = 100):
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    
    for page_name in pages:
        try:
            page = load_with_disambigution(page_name)
        
        except Exception as e:
            print('Skipping page {}'.format(page_name), str(e).strip('\n'))
            continue

        wiki_content.append(f'{page.title}\n{page.content.replace("==", "")}')

    return wiki_content

In [21]:
# для каждого из пяти языков загружаем по 10 рандомных статей

wiki_texts = {}

for lang in langs:
    try:
        wiki_texts[lang] = get_texts_for_lang(lang, 10)
    except Exception as e:
        print('ERROR ON - ', lang, str(e).strip('\n'))
        continue
    
    print(lang, len(wiki_texts[lang]))

es 10
it 10
fr 10
pt 10
ro 10


На этом этапе работы у нас пока что есть словарь вида "язык": "список из десяти текстов на этом языке". Далее нам нужно разделить каждый из текстов на предложения и создать из этих предложений датасет вида "предложение": "язык".

In [88]:
# библиотеки для разделения на предложения и для создания датасета

from nltk import sent_tokenize
import pandas as pd
import re

In [124]:
# сначала создадим список вида [[sent11, lang1], [sent12, lang1], ..., [sent21, lang2], ...]

all_pairs = []

for lang in wiki_texts.keys():
    for text in wiki_texts[lang]:
        sentences = sent_tokenize(text)
        for sentence in sentences:

            # сначала почистим предложения
            sentence = re.sub('\[\d+\]', '', sentence) # удалим сноски в квадратных скобках
            sentence = re.sub('(\\n)|(\n)|(\u200b)|(\\u200b)', ' ', sentence) # удалим все лишние значки
            sentence = re.sub('\d+', ' ', sentence) # удалим вообще все числа
            
            all_pairs.append([sentence, lang])

In [125]:
# теперь запишем это в табличку

data = {'sentence': [pair[0] for pair in all_pairs], 'language': [pair[1] for pair in all_pairs]}

df = pd.DataFrame(data)

In [126]:
df.head()

,sentence,language
0,Er relajo der Loro Er relajo der Loro es una p...,es
1,"La voz de ""Er Loro"" es prestada por el comed...",es
2,Fue estrenada el de junio de en Venezuela.,es
3,Sinopsis La película narra la historia de un ...,es
4,"Reparto Emilio Lovera - (Voz de ""Er Loro"") Lu...",es


На этом этапе у нас есть датасет. Теперь нужно сделать кластеризацию. Я буду использовать алгоритм Bisecting K-Means и делить на 5 кластеров.

In [127]:
# новые библиотеки

from sklearn.cluster import BisectingKMeans
from sklearn.metrics import adjusted_rand_score
from sentence_transformers import SentenceTransformer

In [129]:
# подгружаем модель

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [130]:
grouped_df = df.groupby('language')

In [132]:
# кластеризация

ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['sentence'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # делим на пять кластеров
    cluster = BisectingKMeans(5)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['language'], labels))
    
print(np.mean(ARI))

0.0


Получилось совсем плохо :( Попробуем еще метод Agglomerative Clustering.

In [133]:
from sklearn.cluster import AgglomerativeClustering

In [134]:
# кластеризация

ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['sentence'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # делим на пять кластеров
    cluster = AgglomerativeClustering(5)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['language'], labels))
    
print(np.mean(ARI))

0.0


Всё ещё плохо :( Посмотрим на какие-нибудь неправильно кластеризованные предложения:

In [147]:
# следующие три текста были классифицированы в третью категорию вместо пятой:

labels[43], labels[86], labels[87]

(3, 3, 3)

In [148]:
# достанем их:

texts[43], texts[86], texts[87]

('Avea o greutate în serviciu de   tf și viteză maximă de   km/h.',
 'Aceasta avea o lungime de peste   m, puterea maximă  .  CP și putea atinge o viteză de   km/h.',
 'Capacitatea rezervoarelor este corespunzătoare dimensiunilor unei astfel de locomotive:   m  combustibil,   m  apă,  .  l ulei și   tone nisip.')

# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [194]:
# загружаем библиотеки и корпус

import bz2

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

from lxml import etree
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

In [195]:
# переводим корпус в более удоный формат

corpus = []

for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append([word[0]] + gram_info)
    
    corpus.append(sent_tagged)

In [196]:
# получается такой внешний вид:

corpus[0]

[['«', 'PNCT'],
 ['Школа', 'NOUN', 'inan', 'femn', 'sing', 'nomn'],
 ['злословия', 'NOUN', 'inan', 'neut', 'sing', 'gent'],
 ['»', 'PNCT'],
 ['учит', 'VERB', 'impf', 'tran', 'sing', '3per', 'pres', 'indc'],
 ['прикусить', 'INFN', 'perf', 'tran'],
 ['язык', 'NOUN', 'inan', 'masc', 'sing', 'accs']]

На этом этапе работы у нас есть корпус того вида, как показано выше. Теперь будем смотреть на морфологически омонимичные словоформы. Для этого сначала создадим словарь вида "слово": "\[(разбор 1), (разбор 2), ...]".

In [198]:
# сначала создадим словарь, где будут вообще все, даже совпадающие, разборы слова

total_dictionary = {}

for sentence in corpus:
    for word in sentence:
        form = word[0].lower()
        if form in total_dictionary.keys():
            total_dictionary[form].append(tuple(word[1::]))
        else:
            total_dictionary[form] = []
            total_dictionary[form].append(tuple(word[1::]))

In [199]:
# пример того, что получилось:

total_dictionary['школа']

[('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn'),
 ('NOUN', 'inan', 'femn', 'sing', 'nomn')]

In [200]:
# теперь создадим словарь, в котором будут только неоднозначные (т.е. имеющие > 1 разбора) словоформы, а также их частотности

dict_frequency = {}

for word in total_dictionary.keys():
    new_frequency = len(total_dictionary[word]) # частотность слова
    new_set = set(total_dictionary[word]) # множество его разборов
    if len(new_set) > 1: # если размер множества > 1, т.е. слово имеет больше одного разбора
        dict_frequency[word] = new_frequency # добавляем его и его частотность в словарь подсчета частотностей неоднозначных слов

In [201]:
# находим самую частотную морфологически омонимичную словоформу

max(dict_frequency, key = dict_frequency.get), dict_frequency[max(dict_frequency, key = dict_frequency.get)]

('в', 2059)

Таким образом, самая частотная морфологически омонимичная словоформа - "в", которая встретилась в корпусе 2059 раз. Вот ее разборы:

In [202]:
set(total_dictionary['в'])

{('NOUN', 'inan', 'masc', 'Fixd', 'Abbr', 'sing', 'gent'), ('PREP',)}

Теперь найдем словоформы с самым большим количеством вариантов грамматических разборов.

In [203]:
# создадим словарь, в котором будут все словоформы и количество их грамматических разборов

dict_gram_count = {}

for word in total_dictionary.keys():
    new_set = set(total_dictionary[word]) # множество его разборов
    dict_gram_count[word] = len(new_set) # добавляем его и количество его разборов в словарь

In [204]:
# находим словоформу с самым большим количеством разборов

max(dict_gram_count, key = dict_gram_count.get), dict_gram_count[max(dict_gram_count, key = dict_gram_count.get)]

('сша', 6)

Таким образом, словоформа с наибольшим количеством грамматических разборов - "сша", у которой 6 различных разборов. Вот ее разборы:

In [206]:
set(total_dictionary['сша'])

{('NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'ablt'),
 ('NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'accs'),
 ('NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'datv'),
 ('NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'gent'),
 ('NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'loct'),
 ('NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'nomn')}

Как видно, это просто разборы с указанием разных падежей, поскольку соответствующая лексема имеет одинаковые формы во всех шести падежах.

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением. 

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова) 
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```


In [81]:
# загрузка библиотек

import spacy_udpipe
spacy_udpipe.download('ru')
nlp = spacy_udpipe.load('ru')

from nltk.parse import DependencyGraph
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

from heapq import nlargest

Already downloaded a model for the 'ru' language


Выбранный файл корпуса: ru_syntagrus-ud-train-c.conllu

In [23]:
# загрузка корпуса

trees = []

parsed_sents = open('ru_syntagrus-ud-train-c.conllu').read().split('\n\n')
for sent in parsed_sents:
    tree = [line for line in sent.split('\n') if len(line) > 0 and line[0] != '#']
    trees.append('\n'.join(tree))

In [51]:
# создадим список всех триплов

list_of_triples = []

for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0]
        list_of_triples.append(list(d.triples()))
    except:
        pass

In [52]:
# элементы этого спика выглядят вот так:
list_of_triples[0]

[((None, 'TOP'), 'root', ('родилась', 'VERB')),
 (('родилась', 'VERB'), 'nsubj', ('мать', 'NOUN')),
 (('мать', 'NOUN'), 'det', ('Моя', 'DET')),
 (('мать', 'NOUN'), 'appos', ('Анна', 'PROPN')),
 (('Анна', 'PROPN'), 'punct', (',', 'PUNCT')),
 (('Анна', 'PROPN'), 'flat:name', ('Всеволодовна', 'PROPN')),
 (('Анна', 'PROPN'), 'flat:name', ('Мохова', 'PROPN')),
 (('Мохова', 'PROPN'), 'parataxis', ('Дмитриева', 'PROPN')),
 (('Дмитриева', 'PROPN'), 'punct', ('(', 'PUNCT')),
 (('Дмитриева', 'PROPN'), 'punct', (')', 'PUNCT')),
 (('Анна', 'PROPN'), 'punct', (',', 'PUNCT')),
 (('родилась', 'VERB'), 'obl', ('27', 'ADJ')),
 (('27', 'ADJ'), 'flat', ('марта', 'NOUN')),
 (('27', 'ADJ'), 'nmod', ('года', 'NOUN')),
 (('года', 'NOUN'), 'amod', ('1913', 'ADJ')),
 (('родилась', 'VERB'), 'punct', ('.', 'PUNCT'))]

Далее будем идти по этому списку и вытаскивать пары, связанные определенными тремя отношениями, и считать частотность каждой пары.  
Выбранные отношения:
- det (существительное + determiner)
- case (предлог + существительное)
- iobj (глагол + его непрямой объект)

In [85]:
# программа подсчета пар, связанных выбранными тремя отношениями

# словари, где будут счетчики частотностей; они будут иметь вид "(слово1, слово2)": количество встречаемостей
det_dictionary = {}
case_dictionary = {}
iobj_dictionary = {}

for set_of_triples in list_of_triples:
    for triple in set_of_triples:
        
        # если мы наткнулись на det:
        if triple[1] == 'det':
            new_tuple = (triple[0][0].lower(), triple[2][0].lower())
            if new_tuple in det_dictionary.keys():
                det_dictionary[new_tuple] += 1
            else:
                det_dictionary[new_tuple] = 1

        # если мы наткнулись на case:
        elif triple[1] == 'case':
            new_tuple = (triple[0][0].lower(), triple[2][0].lower())
            if new_tuple in case_dictionary.keys():
                case_dictionary[new_tuple] += 1
            else:
                case_dictionary[new_tuple] = 1
        
        # если мы наткнулись на iobj:
        elif triple[1] == 'iobj':
            new_tuple = (triple[0][0].lower(), triple[2][0].lower())
            if new_tuple in iobj_dictionary.keys():
                iobj_dictionary[new_tuple] += 1
            else:
                iobj_dictionary[new_tuple] = 1

In [82]:
# топ-5 самых частотных пар для det

five_largest_det = nlargest(5, det_dictionary, key = det_dictionary.get)
for pair in five_largest_det:
    print(pair, det_dictionary[pair])

('это', 'все') 79
('числе', 'том') 77
('деле', 'самом') 66
('время', 'все') 41
('время', 'то') 38


In [86]:
# топ-5 самых частотных пар для dobj

five_largest_case = nlargest(5, case_dictionary, key = case_dictionary.get)
for pair in five_largest_case:
    print(pair, case_dictionary[pair])

('году', 'в') 232
('нас', 'у') 168
('меня', 'у') 139
('том', 'о') 137
('том', 'в') 110


In [84]:
# топ-5 самых частотных пар для iobj

five_largest_iobj = nlargest(5, iobj_dictionary, key = iobj_dictionary.get)
for pair in five_largest_iobj:
    print(pair, iobj_dictionary[pair])

('кажется', 'мне') 27
('казалось', 'мне') 12
('хотелось', 'мне') 10
('сказал', 'мне') 10
('позволить', 'себе') 9


Теперь вытащим подзависимые для слов в самых частотных парах - ('это', 'все'), ('году', 'в') и ('кажется', 'мне').

In [97]:
# функция flatten

def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [98]:
# функция get_subtree

def get_subtree(nodes, node):
    
    if not nodes[node]['deps']:
        return [node]
    
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel != 'punct'  # пунктуацию доставать не будем
                         for dep in nodes[node]['deps'][rel]]

In [134]:
# поддеревья для пары ('это', 'все'), связанной отношением det

for tree in trees:
    try:
        d = DependencyGraph(tree)
        for node_i, node in d.nodes.items():
            if (node['word'] == 'это' or node['word'] == 'Это') and 'det' in node['deps']:
                vse = [d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, node['deps']['det'][0])))]
                if 'все' in vse or 'Все' in vse:
                    print('Новое предложение')
                    print('Зависимые для "это":')
                    print([d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])
                    print('Зависимые для "все":')
                    print(vse)
                    print()
                    
    except:
        pass

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['это', 'все']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['это', 'все']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['Это', 'же', 'все']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['за', 'все', 'это']
Зависимые для "все":
['все']

Новое предложение
Зависимые для "это":
['Все', 'это']
Зависимые для "все":
['Все']

Новое предложение
Зависимые для "это":
['все', 'это']
Зависимые 

In [135]:
# поддеревья для пары ('году', 'в'), связанной отношением case

for tree in trees:
    try:
        d = DependencyGraph(tree)
        for node_i, node in d.nodes.items():
            if (node['word'] == 'году' or node['word'] == 'Году') and 'case' in node['deps']:
                v = [d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, node['deps']['case'][0])))]
                if 'в' in v or 'В' in v:
                    print('Новое предложение')
                    print('Зависимые для "году":')
                    print([d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])
                    print('Зависимые для "в":')
                    print(v)
                    print()
                    
    except:
        pass

Новое предложение
Зависимые для "году":
['В', '1937', 'же', 'году']
Зависимые для "в":
['В']

Новое предложение
Зависимые для "году":
['в', '1950', 'году']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году":
['В', '1944', 'году']
Зависимые для "в":
['В']

Новое предложение
Зависимые для "году":
['в', '1932', 'году']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году":
['уже', 'в', '1934', 'году']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году":
['только', 'в', '1977', 'году', 'после', 'принятия', 'брежневской', 'Конституции']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году":
['в', 'том', 'же', '1934', 'году']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году":
['в', '1991', 'году']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году":
['ещё', 'в', '1991', 'году', 'сразу', 'же', 'после', 'объявления', 'Украиной', 'своей', 'независимости']
Зависимые для "в":
['в']

Новое предложение
Зависимые для "году"

In [136]:
# поддеревья для пары ('кажется', 'мне'), связанной отношением iobj

for tree in trees:
    try:
        d = DependencyGraph(tree)
        for node_i, node in d.nodes.items():
            if (node['word'] == 'кажется' or node['word'] == 'Кажется') and 'iobj' in node['deps']:
                mne = [d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, node['deps']['iobj'][0])))]
                if 'мне' in mne or 'Мне' in mne:
                    print('Новое предложение')
                    print('Зависимые для "кажется":')
                    print([d.nodes[i]['word'] for i in sorted(flatten(get_subtree(d.nodes, node_i)))])
                    print('Зависимые для "мне":')
                    print(mne)
                    print()
                    
    except:
        pass

Новое предложение
Зависимые для "кажется":
['мне', 'кажется', 'что', 'самый', 'большой', 'ущерб', 'от', 'того', 'что', 'принес', 'украинский', 'кризис', 'это', 'огромный', 'ущерб', 'доверию', 'между', 'Россией', 'и', 'Западом']
Зависимые для "мне":
['мне']

Новое предложение
Зависимые для "кажется":
['Мне', 'кажется', 'что', 'основной', 'пакет', 'санкций', 'уже', 'введен', 'то', 'есть', 'извне', 'нам', 'сделать', 'хуже', 'экономическими', 'методами', 'уже', 'трудно']
Зависимые для "мне":
['Мне']

Новое предложение
Зависимые для "кажется":
['Мне', 'кажется', 'Путину', 'не', 'до', 'высказываний', 'людей', 'на', 'тему', 'экономики', 'то', 'есть', 'он', 'тоже', 'понимает', 'что', 'это', 'вопрос', 'политический']
Зависимые для "мне":
['Мне']

Новое предложение
Зависимые для "кажется":
['Мне', 'кажется', 'что', 'этой', 'волне', 'будет', 'трудно', 'противостоять']
Зависимые для "мне":
['Мне']

Новое предложение
Зависимые для "кажется":
['Мне', 'кажется', 'что', 'это', 'сейчас', 'довольно', 'т